In [ ]:
!pip install --upgrade pip
!pip install --quiet transformers torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install --quiet pillow evaluate seqeval
!pip install --quiet datasets accelerate
!pip install --quiet git+https://github.com/salaniz/pycocoevalcap.git@master

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install --upgrade transformers --quiet
!pip install --upgrade datasets --quiet
!pip install --upgrade accelerate --quiet

In [25]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
# Define main paths for the project
DRIVE_ROOT = '/content/drive/MyDrive/goPro'           # Root folder in Google Drive for the project
RAW_FOLDER = f"{DRIVE_ROOT}/gopro_images"            # Folder containing original GoPro images
PREPROC_FOLDER = f"{DRIVE_ROOT}/gopro_images_preprocessed"  # Folder to save preprocessed images
OUTPUT_FOLDER = f"{DRIVE_ROOT}/gopro_output"        # Folder to store outputs like captions and evaluation

import os

# Create preprocessing and output folders if they don't exist
os.makedirs(PREPROC_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Display folder paths to confirm setup
print('Drive folders prepared:')
print('RAW_FOLDER =', RAW_FOLDER)
print('PREPROC_FOLDER =', PREPROC_FOLDER)
print('OUTPUT_FOLDER =', OUTPUT_FOLDER)


Drive folders prepared:
RAW_FOLDER = /content/drive/MyDrive/goPro/gopro_images
PREPROC_FOLDER = /content/drive/MyDrive/goPro/gopro_images_preprocessed
OUTPUT_FOLDER = /content/drive/MyDrive/goPro/gopro_output


In [ ]:
from PIL import Image
import os

# Maximum width for resizing images to maintain uniform size
MAX_WIDTH = 640

# List to store any files that cannot be processed
skipped = []

# Loop through all files in the preprocessing folder
for fname in sorted(os.listdir(PREPROC_FOLDER)):
    # Skip files that are not images
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    # Build full paths for input (raw) and output (preprocessed) images
    in_path = os.path.join(RAW_FOLDER, fname)
    out_path = os.path.join(PREPROC_FOLDER, fname)

    try:
        # Open image and convert to RGB (ensures consistent 3-channel format)
        img = Image.open(in_path).convert('RGB')

        # Resize image if its width exceeds MAX_WIDTH while preserving aspect ratio
        if img.width > MAX_WIDTH:
            ratio = MAX_WIDTH / float(img.width)
            new_h = int(img.height * ratio)
            img = img.resize((MAX_WIDTH, new_h), Image.LANCZOS)

        # Save preprocessed image as JPEG with high quality
        img.save(out_path, 'JPEG', quality=95)
        print('Saved', out_path)

    except Exception as e:
        # Store files that failed to process for review
        skipped.append((fname, str(e)))

# Summary after processing all images
if skipped:
    print('Skipped files:', skipped)
else:
    print('Preprocessing complete. All images saved to', PREPROC_FOLDER)


Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0040503.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0061996.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0083423.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0083741.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0083742.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0083893.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0093962.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0094034.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0145306.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0145536.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0156860.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0167284.JPG
Saved /content/drive/MyDrive/goPro/gopro_images_preprocessed/G0229623.JPG
Saved /content/drive/MyDrive/goPro/gop

In [19]:
import pandas as pd
manual = pd.read_csv(os.path.join(OUTPUT_FOLDER, 'manual_captions.csv'))
manual


,file,caption
0,G0587321.JPG,a street with cars parked on the side of it an...
1,G0587322.JPG,a street with cars parked on the side of it
2,G0587323.JPG,a street with cars parked on the side of it
3,G0587324.JPG,a street with cars parked on the side of it an...
4,G0587325.JPG,a street with cars parked on the side of it an...
5,G0587326.JPG,a street filled with lots of motorcycles and cars
6,G0587327.JPG,a truck is parked on side of street
7,G0587328.JPG,a man trying to cross the road
8,G0587329.JPG,a street with cars parked on the side of it
9,G0592403.JPG,an empty street with blue building on left side


In [ ]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
import csv
from PIL import Image
import os

# Load pre-trained BLIP model for image captioning
# Using float16 for reduced memory usage
model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base",
    torch_dtype=torch.float16
)
# Load corresponding processor (handles image preprocessing and tokenization)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

# Set device to GPU if available, else CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# List to store generated captions
captions = []

# Loop through all preprocessed images
for fname in sorted(os.listdir(PREPROC_FOLDER)):
    # Skip non-image files
    if not fname.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue

    # Open and convert image to RGB
    path = os.path.join(PREPROC_FOLDER, fname)
    img = Image.open(path).convert('RGB')

    # Preprocess image and move tensors to the chosen device
    inputs = processor(images=img, return_tensors='pt').to(device)

    # Generate caption using beam search (num_beams=5) and max token length=30
    out = model.generate(**inputs, max_length=30, num_beams=5)

    # Decode generated tokens into a readable string
    caption = processor.decode(out[0], skip_special_tokens=True)

    # Store the filename and generated caption
    captions.append({'file': fname, 'caption': caption})
    print(fname, ':', caption)

# Save all captions to a CSV file
csv_path = os.path.join(OUTPUT_FOLDER, 'captions_baseline.csv')
with open(csv_path, 'w', newline='', encoding='utf-8') as cf:
    writer = csv.writer(cf)
    # Write header row
    writer.writerow(['file', 'caption'])
    # Write each image filename and its corresponding caption
    for r in captions:
        writer.writerow([r['file'], r['caption']])

print('Saved baseline captions to', csv_path)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

G0040503.JPG : a motorcycle parked on the side of a street
G0061996.JPG : a motorcycle parked in front of a building
G0083423.JPG : an empty street in the middle of a city
G0083741.JPG : a motorcycle parked on the side of a road
G0083742.JPG : a motorcycle parked on the side of a street
G0083893.JPG : a group of people riding motorcycles down a street
G0093962.JPG : an empty street in the middle of a city
G0094034.JPG : a motorcycle parked on the side of a road
G0145306.JPG : a street with motorcycles parked on the side of it
G0145536.JPG : a man riding a motorcycle down a street
G0156860.JPG : a street filled with motorcycles and motorcycles
G0167284.JPG : a street with a bunch of signs on the side of it
G0229623.JPG : a woman in a red dress walking down the street
G0291345.JPG : a street with buildings and cars on the side of it
G0291347.JPG : a motorcycle parked on the side of a street
G0291590.JPG : a building with a sign on the front of it
G0302024.JPG : a street with a motorcycle

In [ ]:
# Set sampling interval: pick every Nth caption from the full list
N = 5

# Create a list containing every Nth caption
sampled_captions = [c for i, c in enumerate(captions) if i % N == 0]

# Define CSV path to save sampled captions
sampled_csv = os.path.join(OUTPUT_FOLDER, f'captions_sampled_every_{N}.csv')

# Write sampled captions to CSV
with open(sampled_csv, 'w', newline='', encoding='utf-8') as sf:
    writer = csv.writer(sf)
    # Write header row
    writer.writerow(['file', 'caption'])
    # Write each sampled image filename and its caption
    for r in sampled_captions:
        writer.writerow([r['file'], r['caption']])

# Confirm saving
print(f'Sampled captions saved to {sampled_csv} (every {N}th frame)')


Sampled captions saved to /content/drive/MyDrive/goPro/gopro_output/captions_sampled_every_5.csv (every 5th frame)


In [ ]:
from transformers import pipeline
import os

# Initialize BART summarization pipeline
# device=0 uses GPU if available, -1 uses CPU
summarizer = pipeline(
    'summarization',
    model='facebook/bart-large-cnn',
    device=0 if device=='cuda' else -1
)

# Concatenate all generated captions into a single string
all_text = ' '.join([c['caption'] for c in captions])

# BART has a maximum token limit; split text into smaller chunks if necessary
CHUNK_SIZE = 1000  # characters per chunk
chunks = [all_text[i:i+CHUNK_SIZE] for i in range(0, len(all_text), CHUNK_SIZE)]

# List to store summary of each chunk
summary_parts = []

# Generate summary for each chunk
for chunk in chunks:
    out = summarizer(chunk, max_length=120, min_length=40, do_sample=False)
    summary_parts.append(out[0]['summary_text'])

# Combine all chunk summaries into a single journey summary
journey_summary = ' '.join(summary_parts)

# Save the summary to a text file
summary_path = os.path.join(OUTPUT_FOLDER, 'journey_summary.txt')
with open(summary_path, 'w', encoding='utf-8') as sf:
    sf.write(journey_summary)

# Display the summary and confirm saving
print('\nJourney Summary:\n')
print(journey_summary)
print('\nSaved summary to', summary_path)


Device set to use cuda:0
Your max_length is set to 120, but your input_length is only 66. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=33)



Journey Summary:

A group of people riding motorcycles down a street an empty street in the middle of a city a motorcycle parked on the side of a road a street with motorcycles parked on  it a street filled with motorcycles and motorcycles. a woman in a pink dress walking down the street a man riding a motorcycle down  a street  with a bunch of signs on the  side of it. a building with a sign on the front of it a motorcycle on a street. A street scene with a small store on the side of the road. down a dirt road a red and white striped building next to a road a green building with a sign on theSide of it a motorcycle parked on the Side of a road. a view of a street in a small town a man riding a motorcycle down a street a street filled with lots of vehicles and motorcycles. a street with a tree in the middle of it. A group of people riding motorcycles down a street a street with cars parked on the side of it a street filled with lots of cars and trucks a white car driving down a road n

In [13]:
from PIL import ImageDraw, ImageFont, Image
import pandas as pd
import os

# Load the CSV containing captions
# Expected format: first column = filename, second column = caption
sampled = pd.read_csv('/gopro_output/captions_baseline.csv')

# Use default font for drawing text
font = ImageFont.load_default()

# Folder to save annotated images with captions overlaid
gallery_folder = os.path.join(OUTPUT_FOLDER, 'annotated_gallery')
os.makedirs(gallery_folder, exist_ok=True)

# Iterate over each row in the CSV
for _, row in sampled.iterrows():
    f = row[0]   # filename of the image
    cap = row[1] # generated caption

    # Open preprocessed image and convert to RGB
    ip = os.path.join(PREPROC_FOLDER, f)
    im = Image.open(ip).convert('RGB')

    # Create a drawing context
    draw = ImageDraw.Draw(im)

    # Compute the size of the text to draw a background rectangle
    bbox = draw.textbbox((0, 0), cap, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    padding = 6  # padding around text

    # Draw a white rectangle at the top-left corner to improve readability
    rect_x0, rect_y0 = 10, 10
    rect_x1 = rect_x0 + text_width + padding
    rect_y1 = rect_y0 + text_height + padding
    draw.rectangle([rect_x0, rect_y0, rect_x1, rect_y1], fill=(255, 255, 255))

    # Draw the caption text in black over the rectangle
    draw.text((rect_x0 + padding//2, rect_y0 + padding//2), cap, fill=(0, 0, 0), font=font)

    # Save the annotated image in the gallery folder
    im.save(os.path.join(gallery_folder, f))


/tmp/ipython-input-3390374830.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  f = row[0]   # first column = filename
/tmp/ipython-input-3390374830.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cap = row[1] # second column = caption


In [ ]:
import pandas as pd
import json
import evaluate

# Load manual annotations and generated captions
refs_df = pd.read_csv("/content/drive/MyDrive/goPro/gopro_output/manual_captions.csv")
pred_df = pd.read_csv("/content/drive/MyDrive/goPro/gopro_output/captions_baseline.csv")

# Build dicts keyed by filename
references = {row['file']: [row['caption']] for _, row in refs_df.iterrows()}
hypotheses = {row['file']: [row['caption']] for _, row in pred_df.iterrows()}

# Prepare lists for evaluation (ensuring alignment)
pred_list = [hypotheses[f][0] for f in references.keys()]
ref_list = [[references[f][0]] for f in references.keys()]

# Load Hugging Face metrics
bleu = evaluate.load("bleu")
meteor = evaluate.load("meteor")

# Compute scores
results = {}
results["BLEU"] = bleu.compute(predictions=pred_list, references=ref_list)
results["METEOR"] = meteor.compute(predictions=pred_list, references=ref_list)

# Save the results
eval_path = "/content/drive/MyDrive/goPro/gopro_output/evaluation_results.json"
with open(eval_path, "w") as f:
    json.dump(results, f, indent=4)

print("✅ Evaluation complete. Results saved to:", eval_path)
print(json.dumps(results, indent=4))


✅ Evaluation complete. Results saved to: /content/drive/MyDrive/goPro/gopro_output/evaluation_results.json
{
    "BLEU": {
        "bleu": 0.8625492083332585,
        "precisions": [
            0.937636761487965,
            0.905982905982906,
            0.8825966850828729,
            0.8680445151033387
        ],
        "brevity_penalty": 0.9603270260077138,
        "length_ratio": 0.961093585699264,
        "translation_length": 914,
        "reference_length": 951
    },
    "METEOR": {
        "meteor": 0.9091714265793902
    }
}


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
